# Importing important libraries

In [146]:
import time
import selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
import requests
from bs4 import BeautifulSoup
import re

In [199]:
#connect to times of india webpage
sURL="https://timesofindia.indiatimes.com/"
driver=webdriver.Chrome("/Users/bhupendrakumar/Documents/Personal/chromedriver")
driver.get(sURL)

In [148]:
# Navigate to archives page
link=driver.find_element(By.LINK_TEXT,"Archives")
Archives_Address=link.get_attribute('href')
driver.get(Archives_Address)
time.sleep(3)

In [149]:
# Grab data for Oct 2020
links=driver.find_elements(By.LINK_TEXT,"October")

from datetime import date
d=date.today()
ActiveYear=int(d.strftime("%Y"))
YearToBesearched=2020
Link_Index=ActiveYear-YearToBesearched
driver.get(links[Link_Index-1].get_attribute('href'))

In [150]:
#Store links of days
Days_links=[]
for iDay in range(1,32):
    webaddress=driver.find_element(By.LINK_TEXT,str(iDay)).get_attribute('href')
    Days_links.append(webaddress)

In [152]:
#Store news links
News_links=[]
for iDay in Days_links:
    driver.get(iDay)
    time.sleep(2)
    #Set of news links
    News_Addresses=driver.find_elements_by_xpath("//span[@style='font-family:arial ;font-size:12;color: #006699']")
    
    #News links
    Text_Links=News_Addresses[0].text
    texts=Text_Links.split("\n")
    pattern=r"[a-zA-Z]+"
    for sText in texts:    
        if re.search(pattern,sText):
            webaddress=driver.find_element(By.LINK_TEXT,sText).get_attribute('href')
            News_links.append(webaddress)

In [153]:
len(News_links)

7719

## Defining and intializing lists

In [200]:
Headlines_list=[]
Authors_list=[]
Dates_list=[]
News_list=[]
Verticals_list=[]

In [201]:
#Prev i=6526
i=6526
for sNews_Link in News_links[6526:]:
    i+=1
    print(i)
    driver.get(sNews_Link)
    time.sleep(3)
    try:
        # If error class is present and is empty then add the texts of desired classes.
        driver.find_element_by_xpath("//div[@class='error']")
        if driver.find_element_by_xpath("//div[@class='error']").text =="":
            #Extracting Headlines
            try:
                headline=driver.find_element_by_xpath("//h1[@class='_23498']")
                headline=headline.text
            except ElementNotInteractableException:
                headline=driver.find_element_by_xpath("//h1[@class='heading1']")
                headline=headline.text
            except NoSuchElementException:
                try:
                    headline=driver.find_element_by_xpath("//h1[@class='heading1']")
                    headline=headline.text
                except NoSuchElementException:
                    headline="-"
            Headlines_list.append(headline)

            #Extracting Authors and Dates
            try:
                Author_Date=driver.find_element_by_xpath("//div[@class='_3Mkg- byline']").text
                if "|" in Author_Date:
                    Authors_list.append(Author_Date[:Author_Date.index(" | ")])
                else:
                    Authors_list.append("-")

                if "Oct" not in Author_Date:
                    Dates_list.append("-")
                else:
                    Dates_list.append(Author_Date[Author_Date.index("Oct"):])

            except ElementNotInteractableException:
                Author_Date=driver.find_element_by_xpath("//div[@class='as_byline']")
                Author_Date=Author_Date.text.split('\n')[0].replace("Created:","")
                Authors_list.append(Author_Date[Author_Date.index("-")+2:Author_Date.index("Oct")])
                if "Oct" not in Author_Date:
                    Dates_list.append("-")
                else:
                    Dates_list.append(Author_Date[Author_Date.index("Oct"):])
            except NoSuchElementException:
                try:
                    Author_Date=driver.find_element_by_xpath("//div[@class='as_byline']")
                    Author_Date=Author_Date.text.split('\n')[0].replace("Created:","")
                    
                    if "Oct" not in Author_Date:
                        Dates_list.append("-")
                        Authors_list.append("-")
                    else:
                        Authors_list.append(Author_Date[Author_Date.index("-")+2:Author_Date.index("Oct")])
                        Dates_list.append(Author_Date[Author_Date.index("Oct"):])
                except NoSuchElementException:
                    Dates_list.append("-")
                    Authors_list.append("-")
            #Extracting Description
            try:
                News=driver.find_element_by_xpath("//div[@class='ga-headlines']").text
            except ElementNotInteractableException:
                News=driver.find_element_by_xpath("//div[@class='Normal']").text
                News=News.replace("\n","")
            except NoSuchElementException:
                News=driver.find_element_by_xpath("//div[@class='Normal']").text
                News=News.replace("\n","")
            News_list.append(News)

            #Extracting Verticals
            try:
                Vertical=driver.find_elements_by_xpath("//span[@itemprop='name']")
                if len(Vertical)>=2:
                    Vert=Vertical[1].text
                else:
                    Vert="-"
            except ElementNotInteractableException:
                Vert="-"
            except NoSuchElementException:
                Vert="-"

            Verticals_list.append(Vert)
        else: # If error class exists and text data is peresent then probably news has been removed.
              # Continue from the next page
            continue
    except NoSuchElementException: # If no error class exists, then should have no problem in collecting data
        #Extracting Headlines
            try:
                headline=driver.find_element_by_xpath("//h1[@class='_23498']")
                headline=headline.text
            except ElementNotInteractableException:
                headline=driver.find_element_by_xpath("//h1[@class='heading1']")
                headline=headline.text
            except NoSuchElementException:
                try:
                    headline=driver.find_element_by_xpath("//h1[@class='heading1']")
                    headline=headline.text
                except NoSuchElementException:
                    headline="-"
            Headlines_list.append(headline)

            #Extracting Authors and Dates
            try:
                Author_Date=driver.find_element_by_xpath("//div[@class='_3Mkg- byline']").text
                if "|" in Author_Date:
                    Authors_list.append(Author_Date[:Author_Date.index(" | ")])
                else:
                    Authors_list.append("-")

                if "Oct" not in Author_Date:
                    Dates_list.append("-")
                else:
                    Dates_list.append(Author_Date[Author_Date.index("Oct"):])

            except ElementNotInteractableException:
                Author_Date=driver.find_element_by_xpath("//div[@class='as_byline']")
                Author_Date=Author_Date.text.split('\n')[0].replace("Created:","")
                Authors_list.append(Author_Date[Author_Date.index("-")+2:Author_Date.index("Oct")])
                if "Oct" not in Author_Date:
                    Dates_list.append("-")
                else:
                    Dates_list.append(Author_Date[Author_Date.index("Oct"):])
            except NoSuchElementException:
                try:
                    Author_Date=driver.find_element_by_xpath("//div[@class='as_byline']")
                    Author_Date=Author_Date.text.split('\n')[0].replace("Created:","")
                    
                    if "Oct" not in Author_Date:
                        Authors_list.append("-")
                        Dates_list.append("-")
                    else:
                        Authors_list.append(Author_Date[Author_Date.index("-")+2:Author_Date.index("Oct")])
                        Dates_list.append(Author_Date[Author_Date.index("Oct"):])
                except NoSuchElementException:
                    Dates_list.append("-")
                    Authors_list.append("-")
            #Extracting Description
            try:
                News=driver.find_element_by_xpath("//div[@class='ga-headlines']").text
            except ElementNotInteractableException:
                News=driver.find_element_by_xpath("//div[@class='Normal']").text
                News=News.replace("\n","")
            except NoSuchElementException:
                News=driver.find_element_by_xpath("//div[@class='Normal']").text
                News=News.replace("\n","")
            News_list.append(News)

            #Extracting Verticals
            try:
                Vertical=driver.find_elements_by_xpath("//span[@itemprop='name']")
                if len(Vertical)>=2:
                    Vert=Vertical[1].text
                else:
                    Vert="-"
            except ElementNotInteractableException:
                Vert="-"
            except NoSuchElementException:
                Vert="-"

            Verticals_list.append(Vert)

6527
6528
6529
6530
6531
6532
6533
6534
6535
6536
6537
6538
6539
6540
6541
6542
6543
6544
6545
6546
6547
6548
6549
6550
6551
6552
6553
6554
6555
6556
6557
6558
6559
6560
6561
6562
6563
6564
6565
6566
6567
6568
6569
6570
6571
6572
6573
6574
6575
6576
6577
6578
6579
6580
6581
6582
6583
6584
6585
6586
6587
6588
6589
6590
6591
6592
6593
6594
6595
6596
6597
6598
6599
6600
6601
6602
6603
6604
6605
6606
6607
6608
6609
6610
6611
6612
6613
6614
6615
6616
6617
6618
6619
6620
6621
6622
6623
6624
6625
6626
6627
6628
6629
6630
6631
6632
6633
6634
6635
6636
6637
6638
6639
6640
6641
6642
6643
6644
6645
6646
6647
6648
6649
6650
6651
6652
6653
6654
6655
6656
6657
6658
6659
6660
6661
6662
6663
6664
6665
6666
6667
6668
6669
6670
6671
6672
6673
6674
6675
6676
6677
6678
6679
6680
6681
6682
6683
6684
6685
6686
6687
6688
6689
6690
6691
6692
6693
6694
6695
6696
6697
6698
6699
6700
6701
6702
6703
6704
6705
6706
6707
6708
6709
6710
6711
6712
6713
6714
6715
6716
6717
6718
6719
6720
6721
6722
6723
6724
6725
6726


In [202]:
print(len(News_list))
print(len(Headlines_list))
print(len(Authors_list))
print(len(Dates_list))
print(len(Verticals_list))

1193
1193
1193
1193
1193


In [189]:
Verticals_list

['CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'POLITICS',
 'POLITICS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY NEWS',
 'CITY N

In [185]:
Headlines_list.pop()
Authors_list.pop()
Dates_list.pop()

'-'

## Storing Data in CSV file

In [203]:
#Declaring the dataframe
Scrapped_data=pd.DataFrame()

#Initializing columns in the new data frame
Scrapped_data['Date']=Dates_list
Scrapped_data['Author']=Authors_list
Scrapped_data['Vertical']=Verticals_list
Scrapped_data['Headline']=Headlines_list
Scrapped_data['Description']=News_list

In [204]:
#Storing data in csv file, in m ultiple iterations
Scrapped_data.to_csv("/Users/bhupendrakumar/Documents/Personal/Internship/News Classification And Analysis/Oct2020_data3.csv")

In [207]:
# Merge different data files collected during extraction process
Data_1=pd.read_csv("/Users/bhupendrakumar/Documents/Personal/Internship/News Classification And Analysis/Oct2020_data.csv")
Data_2=pd.read_csv("/Users/bhupendrakumar/Documents/Personal/Internship/News Classification And Analysis/Oct2020_data2.csv")
Data_3=pd.read_csv("/Users/bhupendrakumar/Documents/Personal/Internship/News Classification And Analysis/Oct2020_data3.csv")
Data_4=pd.read_csv("/Users/bhupendrakumar/Documents/Personal/Internship/News Classification And Analysis/Oct2020_data1.csv")
#Data_5=pd.read_csv("/Users/bhupendrakumar/Documents/Personal/Internship/News Classification And Analysis/July2020_data5.csv")

In [208]:
Final_data=pd.concat([Data_1,Data_2,Data_3,Data_4],ignore_index=True)

In [209]:
# It has 7712 rows and 6 columns
Final_data.shape

(7717, 6)

In [210]:
Final_data.columns

Index(['Unnamed: 0', 'Date', 'Author', 'Vertical', 'Headline', 'Description'], dtype='object')

In [211]:
# Drop unnamed column
Final_data.drop('Unnamed: 0',inplace=True,axis=1)

In [212]:
# Observing first 20 rows of the collected data
Final_data.head(20)

Date                      Author       Vertical  \
0   Oct 1, 2020, 00:00 IST   TNNShamayita.Chakraborty   Entertainment   
1   Oct 1, 2020, 00:00 IST         Joyeeta Chakravorty      CITY NEWS   
2   Oct 1, 2020, 00:00 IST           TIMESOFINDIA.COM       Lifestyle   
3   Oct 1, 2020, 09:07 IST   TIMESOFINDIA.COMUpdated:       Lifestyle   
4   Oct 1, 2020, 00:51 IST  TNNNeha MaheshwriUpdated:              TV   
5   Oct 1, 2020, 00:07 IST                FLAVIO LOPES    SPORTS NEWS   
6   Oct 1, 2020, 00:12 IST         Debabrata Mohapatra     INDIA NEWS   
7   Oct 1, 2020, 00:13 IST            TNNVinay Lokesh   Entertainment   
8   Oct 1, 2020, 00:18 IST                         PTI     INDIA NEWS   
9   Oct 1, 2020, 00:19 IST           TIMESOFINDIA.COM   Entertainment   
10  Oct 1, 2020, 00:21 IST                        TNN       Lifestyle   
11  Oct 1, 2020, 00:42 IST                         TNN      CITY NEWS   
12  Oct 1, 2020, 00:42 IST                         TNN      CITY NEWS   
13  Oct 1, 2020, 01:00 IST             TNNMeera Vohra   Entertainment   
14  Oct 1, 2020, 01:00 IST           Abhimanyu Mishra   Entertainment   
15  Oct 1, 2020, 01:00 IST          TNNOnkar Kulkarni   Entertainment   
16  Oct 1, 2020, 01:00 IST           TNNRuman Ganguly   Entertainment   
17  Oct 1, 2020, 01:00 IST                        TNN   Entertainment   
18  Oct 1, 2020, 01:00 IST                         TNN      CITY NEWS   
19  Oct 1, 2020, 01:01 IST                              Entertainment   

                                             Headline  \
0   Badshah’s Genda Phool now gets a Bengali makeover   
1   Bengaluru poised for a no-frills Durga Puja ce...   
2   Adequate levels of Vitamin D can prevent the s...   
3   Diabetes and peanuts: Here is the right way to...   
4   EXCLUSIVE: 'Dil Toh Happy Hai Ji' actor Ansh B...   
5            Goan boxers left to punch without gloves   
6   NCRB places Odisha on top in child pornography...   
7   Abhishek Das says he lost out on a film role b...   
8   Its for those against Bengaluru's security to ...   
9   Photos: Kartik Aaryan keeps it cool and casual...   
10  Dr Vali says millets take six hours to digest,...   
11  Raipur youngsters give a fillip to traditional...   
12  Raipur youngsters give a fillip to traditional...   
13  I was elated rather than sceptical about outdo...   
14  Bollywood has always been a soft target: Supri...   
15  Sonu Sood: People will question your efforts, ...   
16  Arpita: For me, every b’day is about a new beg...   
17  Raipur's LGBTQIA community celebrates one year...   
18  Raipur's LGBTQIA community celebrate one year ...   
19  Guru Randhawa and Dhvani Bhanushali come toget...   

                                          Description  
0   In March this year, Boroloker Bitilo – a popul...  
1   Every year, Durga Puja is celebrated in a gran...  
2   A recent study conducted by Boston University ...  
3   Patients suffering from diabetes are generally...  
4   Actor Ansh Bagri, who was last seen in the sho...  
5   Panaji: The Goa Boxing Association knows all a...  
6   BHUBANESWAR: Odisha reported the highest numbe...  
7   Television and film actor Abhishek Das was one...  
8   BENGALURU: Newly appointed BJP Yuva Morcha chi...  
9   On Wednesday evening, actor Kartik Aaryan deci...  
10  A simple lifestyle, with the right kind of foo...  
11  While the rage for hampers and dry-fruit boxes...  
12  While the rage for hampers and dry-fruit boxes...  
13  Actor Hemant Pandey aka Pandeyji of teleshow O...  
14  As Ahmedabad Times visited the set of Siddhart...  
15  Sonu Sood, who helped stranded migrant workers...  
16  Arpita feels that we are at a juncture of aver...  
17  Draped in rainbow colours, Raipur's LGBTQIA co...  
18  Draped in rainbow colours, Raipur's LGBTQIA co...  
19  After their first collaboration two years ago ...

In [213]:
#Store Final data in csv format
Final_data.to_csv("/Users/bhupendrakumar/Documents/Personal/Internship/News Classification And Analysis/Oct2020_data(TOI).csv")